In [1]:
import keras
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pandas import datetime
import math, time
import itertools
from sklearn import preprocessing
import datetime
from operator import itemgetter
from sklearn.metrics import mean_squared_error
from math import sqrt
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.recurrent import LSTM
import QUANTAXIS as QA
# 获取全市场的股票代码信息列表
QA_code_list = QA.QA_fetch_stock_list_adv()
#QA_code_list
#QA全市场code列表
QA_code = QA_code_list.code.tolist() 
#QA_code
#QA全市场name列表
QA_name = QA_code_list.name.tolist() 
#QA_name 


Using TensorFlow backend.


In [2]:
#训练起始时间
start_date='2015-06-01'
end_date='2017-06-01'
#inst = D.instruments(start_date, end_date, market='CN_STOCK_A')
#print(inst)
instruments = ['600519']
#instruments = QA_code
hists = QA.QA_fetch_stock_day_adv(instruments, start_date, end_date).data
hist = hists


In [3]:
## load data function
def load_data(stock, seq_len):
    # stock 是一个input的dataframe
    amount_of_features = len(stock.columns)
    data = stock.as_matrix() #pd.DataFrame(stock)
    sequence_length = seq_len + 1
    result = []
    for index in range(len(data) - sequence_length):
        result.append(data[index: index + sequence_length])

    result = np.array(result)
    row = round(0.9 * result.shape[0])
    train = result[:int(row), :]
    x_train = train[:, :-1]
    y_train = train[:, -1][:,-1]
    x_test = result[int(row):, :-1]
    y_test = result[int(row):, -1][:,-1]

    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], amount_of_features))
    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], amount_of_features))  

    return [x_train, y_train, x_test, y_test]

def build_model(layers):
    model = Sequential()

    model.add(LSTM(
        input_dim=layers[0],
        output_dim=layers[1],
        return_sequences=True))
    model.add(Dropout(0.2))

    model.add(LSTM(
        layers[2],
        return_sequences=False))
    model.add(Dropout(0.2))

    model.add(Dense(
        output_dim=layers[2]))
    model.add(Activation("linear"))

    start = time.time()
    model.compile(loss="mse", optimizer="rmsprop",metrics=['accuracy'])
    print("Compilation Time : ", time.time() - start)
    return model

def build_model2(layers):
        d = 0.2
        model = Sequential()
        model.add(LSTM(128, input_shape=(layers[1], layers[0]), return_sequences=True))
        model.add(Dropout(d))
        model.add(LSTM(64, input_shape=(layers[1], layers[0]), return_sequences=False))
        model.add(Dropout(d))
        model.add(Dense(16,init='uniform',activation='relu'))        
        model.add(Dense(1,init='uniform',activation='relu'))
        model.compile(loss='mse',optimizer='adam',metrics=['accuracy'])
        return model

In [4]:
data=QA.QA_fetch_index_day_adv('000001','2010-01-01','2019-03-01')
data.plot()
plt.show()

QUANTAXIS>> The Pic has been saved to your path: ./QA_index_day_None_.html


In [5]:
#data = hist
#data.plot()
#plt.show()

QUANTAXIS>> The Pic has been saved to your path: ./QA_index_day_None_.html


In [6]:
used_data=pd.concat([data.open,data.high,data.close],axis=1)

In [7]:
window = 5
X_train, y_train, X_test, y_test = load_data(used_data[::-1], window)

/Volumes/QuantOsUSB/QuantOsUSB/kivy.app/Contents/Resources/venv/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """


In [8]:
X_train

array([[[2954.4 , 2994.01, 2994.01],
        [2950.05, 2965.3 , 2940.95],
        [2939.93, 2997.49, 2953.82],
        [2969.1 , 2995.68, 2941.52],
        [2838.39, 2961.8 , 2961.28]],

       [[2950.05, 2965.3 , 2940.95],
        [2939.93, 2997.49, 2953.82],
        [2969.1 , 2995.68, 2941.52],
        [2838.39, 2961.8 , 2961.28],
        [2749.46, 2804.23, 2804.23]],

       [[2939.93, 2997.49, 2953.82],
        [2969.1 , 2995.68, 2941.52],
        [2838.39, 2961.8 , 2961.28],
        [2749.46, 2804.23, 2804.23],
        [2759.94, 2794.01, 2751.8 ]],

       ...,

       [[2908.87, 2921.27, 2898.14],
        [2927.26, 2939.05, 2911.41],
        [2928.3 , 2933.41, 2927.08],
        [2850.33, 2923.77, 2922.95],
        [2798.74, 2850.42, 2841.04]],

       [[2927.26, 2939.05, 2911.41],
        [2928.3 , 2933.41, 2927.08],
        [2850.33, 2923.77, 2922.95],
        [2798.74, 2850.42, 2841.04],
        [2837.61, 2854.7 , 2810.95]],

       [[2928.3 , 2933.41, 2927.08],
        [2850.3

In [ ]:
model = build_model2([3,window,1])

/Volumes/QuantOsUSB/QuantOsUSB/kivy.app/Contents/Resources/venv/lib/python3.6/site-packages/ipykernel_launcher.py:54: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(16, activation="relu", kernel_initializer="uniform")`
/Volumes/QuantOsUSB/QuantOsUSB/kivy.app/Contents/Resources/venv/lib/python3.6/site-packages/ipykernel_launcher.py:55: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="relu", kernel_initializer="uniform")`


In [ ]:
model.fit(
    X_train,
    y_train,
    batch_size=512,
    nb_epoch=500,
    validation_split=0.1,
    verbose=0)

/Volumes/QuantOsUSB/QuantOsUSB/kivy.app/Contents/Resources/venv/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  import sys


In [ ]:
# print(X_test[-1])
diff=[]
ratio=[]
p = model.predict(X_test)
for u in range(len(y_test)):
    pr = p[u][0]
    ratio.append((y_test[u]/pr)-1)
    diff.append(abs(y_test[u]- pr))
    #print(u, y_test[u], pr, (y_test[u]/pr)-1, abs(y_test[u]- pr))

In [ ]:
import matplotlib.pyplot as plt2
plt2.figure(figsize=(16,16)) #图表长宽量 16，16
plt2.grid(True)
plt2.plot(p,color='red', label='prediction') #预测
plt2.plot(y_test,color='blue', label='y_test')
plt2.legend(loc='upper left')
plt2.show()